# 네이버 매장정보 크롤링

### 네이버 매장 존재 여부 확인 크롤링에서 만든 파일을 사용합니다

In [1]:
# 웹 드라이버 설정
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 

# 대기 관련 라이브러리
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 

# 예외 처리 관련 라이브러리
from selenium.common.exceptions import TimeoutException, NoSuchElementException  

# 웹 요소 찾기 관련 라이브러리
from selenium.webdriver.common.by import By  
from selenium.webdriver.support.ui import Select  
from selenium.webdriver.common.keys import Keys  

# 그 외 
import time 
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup 
import numpy as np  
import pandas as pd 
import re  
from tqdm import tqdm  # 반복문 진행 상황 시각화 모듈
import zipfile
import os

In [2]:
# # webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
# driver = webdriver.Chrome(ChromeDriverManager().install())


# # webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()
driver = webdriver.Chrome()


# 주소 이동
url = 'https://map.naver.com/'
driver.get(url)
time.sleep(1)

In [3]:
# 데이터 불러오기
original_res_df = pd.read_excel('combined_file.xlsx')
res_df = original_res_df.copy()
res_df = res_df.loc[res_df['검색여부'] == '가능'].reset_index(drop=True)
res_df.drop('Unnamed: 0', axis=1, inplace=True)


# 컬럼 순서 바꾸기 -> 보기 좋으라고 바꿈
res_df['검색어'] = res_df['상세주소'] + " " + res_df['사업장명']
column_order = ['검색어', '업태구분명', '사업장명', '사업장명_원본', '시도', '지역구', '상세주소', '소재지전체주소', '도로명전체주소', '좌표정보(x)', '좌표정보(y)', '검색여부', '영업상태명']
res_df = res_df[column_order]
res_df.tail()

,검색어,업태구분명,사업장명,사업장명_원본,시도,지역구,상세주소,소재지전체주소,도로명전체주소,좌표정보(x),좌표정보(y),검색여부,영업상태명
8291,"도곡로 401, 간코",일식,간코,간코 롯데백화점강남점,서울특별시,강남구,"도곡로 401,",서울특별시 강남구 대치동 937 롯데백화점,"서울특별시 강남구 도곡로 401, 롯데백화점 지하1층 (대치동)",204669.543367,443873.621189,가능,영업/정상
8292,"압구정로 108, 오토김밥",분식,오토김밥,오토김밥 신사점,서울특별시,강남구,"압구정로 108,",서울특별시 강남구 신사동 528-3 덕산빌딩,"서울특별시 강남구 압구정로 108, 덕산빌딩 지상1층 10호 (신사동)",201749.456414,446742.828124,가능,영업/정상
8293,"압구정로42길 36, 마들마들",한식,마들마들,마들마들,서울특별시,강남구,"압구정로42길 36,",서울특별시 강남구 신사동 635-14 금광아크존1,"서울특별시 강남구 압구정로42길 36, 지상1층 101호 (신사동, 금광아크존1)",202990.870783,447146.555568,가능,영업/정상
8294,"언주로151길 21, 세바",경양식,세바,세바(Sevva),서울특별시,강남구,"언주로151길 21,",서울특별시 강남구 신사동 629-1,"서울특별시 강남구 언주로151길 21, 지상1층 (신사동)",202731.421127,446609.460134,가능,영업/정상
8295,"논현로161길 37, 구구구",경양식,구구구,구구구 (GOOGOOGOO),서울특별시,강남구,"논현로161길 37,",서울특별시 강남구 신사동 552-5,"서울특별시 강남구 논현로161길 37, 동쪽 지상1층 (신사동)",202153.410454,446748.562578,가능,영업/정상


In [4]:
# 가져올 정보들에 대한 리스트 지정
# 가게명, 업태명, 주소, 주력메뉴(맨첫번째메뉴) + 가격까지, 방문자리뷰수,블로그리뷰수

restaurant_name_list = []
category_name_list = []
address_list = []
menu_list = []
price_list = []
visitor_review_list =[]
blog_review_list =[]

In [6]:
# 크롤링 시작합니다


for i in range(len(res_df)):
    # 검색어를 네이버 url에 포함시켜 이동 + 검색 
    
    name = res_df['검색어'][i]
    driver.get('https://map.naver.com/p/search/{}'.format(name))

        
    time.sleep(3)  

    ## 화면이 안넘어가고 검색결과에 머물러 있을 때는 entryIframe 이 없음
    ## 그래서 검색어 쳤을때 entryIframe 있으면 --> 화면이 넘어갔다는 뜻이고 
    ## 없으면 아직 안넘어간 뜻이니깐 searchIframe 에서 클릭해서 넘어가야함 .
    try :
        if driver.find_elements(By.ID,'entryIframe') :
            entryIframe = driver.find_element(By.ID,'entryIframe')
            driver.switch_to.frame(entryIframe)
            
            time.sleep(3) 
            
            # 가게 이름, 변수로 지정
            # 메뉴와 가격이 존재하는지의 여부를 고려 
            # 매장마다 메뉴를 보여주는 테마가 2가지 존재하기 때문에 이를 고려

            restaurant_name = driver.find_element(By.CLASS_NAME,'Fc1rA').text
            category_name = driver.find_element(By.CLASS_NAME,'DJJvD').text
            address = driver.find_element(By.CLASS_NAME,'LDgIH').text
            if driver.find_elements(By.CLASS_NAME,'ihmWt') :
                menu = driver.find_element(By.CLASS_NAME,'ihmWt').text
                price = driver.find_element(By.CLASS_NAME,'mkBm3').text
            elif driver.find_elements(By.CLASS_NAME,'VQvNX') :
                menu = driver.find_element(By.CLASS_NAME,'VQvNX').text
                price = driver.find_element(By.CLASS_NAME,'Yrsei').text
            else : 
                menu = '메뉴없음'
                price = "가격없음"
            
            # 리뷰 긁어오기 
            # PXMot 클래스 요소들을 모두 찾음
            # PXMot 는 리뷰의 클래스 이름인데 별점, 방문자리뷰, 블로그리뷰 총 3개가 존재
            pxmot_elements = driver.find_elements(By.CLASS_NAME, 'PXMot')
            
            visitor_review = 0
            blog_review = 0
            # PXMot 가 존재할 때 (별점 방문자 블로그 중  뭐라도 존재)
            if pxmot_elements:
                for element in pxmot_elements:
                    element_text = element.text
                    if element_text.startswith('별점'):
                        # 별점이 있으면 다음 요소 검사
                        continue
                    elif element_text.startswith('방문자리뷰'):
                        # 방문자 리뷰가 존재하는 경우
                        review_text = element_text.replace('방문자리뷰', '')
                        # 쉼표(,) 제거 후 정수로 변환
                        try : 
                            visitor_review = int(review_text.replace(',', '')) if ',' in review_text else int(review_text)
                        except ValueError :
                            visitor_review = 0
                    elif element_text.startswith('블로그리뷰'):
                        # 블로그 리뷰가 존재하는 경우
                        review_text = element_text.replace('블로그리뷰', '')
                        # 쉼표(,) 제거 후 정수로 변환
                        try :
                            blog_review = int(review_text.replace(',', '')) if ',' in review_text else int(review_text)
                        except ValueError :
                            blog_review = 0
            # PXMot 가 없을 때(별점 방문자 블로그 중  뭐라도 존재x)
            else  : 
                visitor_review = 0
                blog_review = 0      
        else: 
            # searchIframe 프레임 
            # searchIframe 프레임으로 가야하는 경우는 검색했을 때 결과가 바로 안나오고 식당리스트가 먼저나오는 경우
            # 그래서 searchIFrame에서 첫번쨰 식당을 선택하고 그다음에 entryIFrame으로 들어가야함
            searchIframe = driver.find_element(By.ID,'searchIframe')
            driver.switch_to.frame(searchIframe)


            ## 클릭 
            driver.find_element(By.CLASS_NAME,'YwYLL').click()
            
            # 기본프레임 
            time.sleep(1)
            driver.switch_to.default_content()
        

            # entryIframe 프레임 
            entryIframe = driver.find_element(By.ID,'entryIframe')
            driver.switch_to.frame(entryIframe)

            time.sleep(3) 
            # 가게 이름, 변수로 지정
            restaurant_name = driver.find_element(By.CLASS_NAME,'Fc1rA').text
            category_name = driver.find_element(By.CLASS_NAME,'DJJvD').text
            address = driver.find_element(By.CLASS_NAME,'LDgIH').text

            # 메뉴와 가격이 존재하는지의 여부를 고려 
            # 매장마다 메뉴를 보여주는 테마가 2가지 존재하기 때문에 이를 고려

            if driver.find_elements(By.CLASS_NAME,'ihmWt') :
                menu = driver.find_element(By.CLASS_NAME,'ihmWt').text
                price = driver.find_element(By.CLASS_NAME,'mkBm3').text
            elif driver.find_elements(By.CLASS_NAME,'VQvNX') :
                menu = driver.find_element(By.CLASS_NAME,'VQvNX').text
                price = driver.find_element(By.CLASS_NAME,'Yrsei').text
            else : 
                menu = '메뉴없음'
                price = "가격없음"

            # 리뷰 수 긁어오기
            # PXMot 클래스 요소들을 모두 찾음
            # PXMot 는 리뷰의 클래스 이름인데 별점, 방문자리뷰, 블로그리뷰 총 3개가 존재
            pxmot_elements = driver.find_elements(By.CLASS_NAME, 'PXMot')
        
            # PXMot 가 존재할 때 (별점 방문자 블로그 중  뭐라도 존재)
            if pxmot_elements:
                for element in pxmot_elements:
                    element_text = element.text
                    if element_text.startswith('별점'):
                        # 별점이 있으면 다음 요소 검사
                        continue
                    elif element_text.startswith('방문자리뷰'):
                        # 방문자 리뷰가 존재하는 경우
                        review_text = element_text.replace('방문자리뷰', '')
                        # 쉼표(,) 제거 후 정수로 변환
                        try : 
                            visitor_review = int(review_text.replace(',', '')) if ',' in review_text else int(review_text)
                        except ValueError :
                            visitor_review = 0
                    elif element_text.startswith('블로그리뷰'):
                        # 블로그 리뷰가 존재하는 경우
                        review_text = element_text.replace('블로그리뷰', '').strip()
                        # 쉼표(,) 제거 후 정수로 변환
                        try :
                            blog_review = int(review_text.replace(',', '')) if ',' in review_text else int(review_text)
                        except ValueError :
                            blog_review = 0
            # PXMot 가 없을 때(별점 방문자 블로그 중  뭐라도 존재x)
            else  : 
                visitor_review = 0
                blog_review = 0        
    ## 예외처리 : vscode 상태가 안좋아서 그냥 검색이 안된경우 이거나 진짜 없는 식당일 때.
    except Exception as e : 
        restaurant_name = name 
        category_name = '전처리필요'
        address = '전처리필요'
        menu = '메뉴없음'
        price = "가격없음"
        visitor_review = 0
        blog_review = 0

        print(f"에러 메시지: {str(e)}")


    ## 리스트에 추가
    restaurant_name_list.append(restaurant_name)
    category_name_list.append(category_name)
    address_list.append(address)
    menu_list.append(menu)
    price_list.append(price)
    visitor_review_list.append(visitor_review)
    blog_review_list.append(blog_review)

    
    print('식당 이름:',restaurant_name)
    print('식당 카테고리:',category_name)
    print('주소:',address)
    print('대표 메뉴:',menu)
    print('가격:',price)
    print('방문자리뷰 : ',visitor_review)
    print('블로그리뷰 : ',blog_review)
    print('-----------------------------')
res_info_df = pd.DataFrame({'식당이름':restaurant_name_list ,
                             '업태구분' :category_name_list ,
                             '주소' :address_list ,
                             '메뉴': menu_list,
                             '가격':price_list,
                             '방문자리뷰':visitor_review_list,
                             '블로그리뷰':blog_review_list,
                             })


## 결과 데이터에 검색어 칼럼 추가
## 추후에 인덱스로 쓸 예정이기 떄문

res_info_df['검색어'] = res_df['검색어']
    

식당 이름: 정식당
식당 카테고리: 한식
주소: 서울 강남구 강남대로114길 18 . 1층 정식당
대표 메뉴: 소불고기주물럭
가격: 16,000원
방문자리뷰 :  11
블로그리뷰 :  0
-----------------------------
식당 이름: 채선당 샤브보트 강남역점
식당 카테고리: 샤브샤브
주소: 서울 강남구 강남대로 378 . 지상1층
대표 메뉴: 소고기 샤브샤브
가격: 12,900원
방문자리뷰 :  85
블로그리뷰 :  64
-----------------------------
식당 이름: 수아당 갤러리아백화점 명품관 WEST점
식당 카테고리: 김밥
주소: 서울 강남구 압구정로 343
대표 메뉴: 비빔면
가격: 10,000원
방문자리뷰 :  10
블로그리뷰 :  20
-----------------------------
식당 이름: 알찬한끼
식당 카테고리: 종합분식
주소: 서울 강남구 역삼로 204 삼환아르누보 지하1층 107호
대표 메뉴: 알찬세트3
가격: 9,500원
방문자리뷰 :  0
블로그리뷰 :  0
-----------------------------
식당 이름: 세계관 삼성본점
식당 카테고리: 한식
주소: 서울 강남구 삼성로96길 11 2층
대표 메뉴: 투뿔 한우 곱도리탕
가격: 32,000원
방문자리뷰 :  62
블로그리뷰 :  18
-----------------------------
식당 이름: 분식을품다 한티점
식당 카테고리: 종합분식
주소: 서울 강남구 역삼로 409 1층 102호
대표 메뉴: 쌀 떡볶이
가격: 5,900원
방문자리뷰 :  31
블로그리뷰 :  13
-----------------------------
식당 이름: 식당민홍
식당 카테고리: 일식당
주소: 서울 강남구 논현로146길 11 2층
대표 메뉴: 오마카세 코스
가격: 122,000원
방문자리뷰 :  27
블로그리뷰 :  171
-----------------------------
식당 이름: 디어
식당 카테고리: 카페,디저트
주소: 서울 강남구 논현로

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=122.0.6261.95)
Stacktrace:
	GetHandleVerifier [0x00007FF7CC8DAD22+56930]
	(No symbol) [0x00007FF7CC84F622]
	(No symbol) [0x00007FF7CC7042E5]
	(No symbol) [0x00007FF7CC6F191F]
	(No symbol) [0x00007FF7CC6F1490]
	(No symbol) [0x00007FF7CC706411]
	(No symbol) [0x00007FF7CC7878B9]
	(No symbol) [0x00007FF7CC76BA43]
	(No symbol) [0x00007FF7CC73D438]
	(No symbol) [0x00007FF7CC73E4D1]
	GetHandleVerifier [0x00007FF7CCC56AAD+3709933]
	GetHandleVerifier [0x00007FF7CCCAFFED+4075821]
	GetHandleVerifier [0x00007FF7CCCA817F+4043455]
	GetHandleVerifier [0x00007FF7CC979756+706710]
	(No symbol) [0x00007FF7CC85B8FF]
	(No symbol) [0x00007FF7CC856AE4]
	(No symbol) [0x00007FF7CC856C3C]
	(No symbol) [0x00007FF7CC8468F4]
	BaseThreadInitThunk [0x00007FFD3C97257D+29]
	RtlUserThreadStart [0x00007FFD3D98AA58+40]


만약에 에러나면 에러난 식당의 리뷰가 그 전 식당의 리뷰수를 따라가더라 

In [7]:
## 잘 긁어왔나 확인합니다

res_info_df.head()

,식당이름,업태구분,주소,메뉴,가격,리뷰,검색어
0,남오토코 역삼점,이자카야,서울 강남구 논현로85길 29 2층,꼬치8종,"35,500원",300,"논현로85길 29, 남오토코"
1,뢰벤돈까스,양식,서울 강남구 테헤란로 124 삼원타워 지하1층 B102호,등심돈까스,"9,000원",302,테헤란로 124 뢰벤 돈까스
2,생강김밥 본점,김밥,서울 강남구 남부순환로 2738 상일빌딩,문어김밥,"4,800원",2032,남부순환로 2738 생강김밥
3,삼성원조양곱창,"곱창,막창,양",서울 강남구 학동로101길 7 승창빌딩 1층,특양,"30,000원",315,"학동로101길 7, 삼성양곱창"
4,152st whisky bar,바(BAR),서울 강남구 테헤란로81길 62-1 2층 152st whisky bar,보테가골드+치즈플레이트,"300,000원",1,"테헤란로81길 62-1, 152st"


In [ ]:
## 엑셀파일에 잘 저장합니다

res_info_df.to_excel('매장정보.xlsx')


# --------------------------------------------------------------------------------------

# 여기서부턴 파일 옮겨서 돌려야 할 듯 합니다

# 업태가 뭐있는지 몰라서입니다  + 리뷰도 다른 파일에서 취합하는ㄱ네 맞는듯 싶습니다 


In [8]:
res_info_df['업태구분'].unique()

array(['이자카야', '양식', '김밥', '곱창,막창,양', '바(BAR)', '백반,가정식', '피자', '카페,디저트',
       '육류,고기요리', '돼지고기구이', '요리주점', '한식뷔페', '뷔페', '한식', '돈가스', '장소대여',
       '일식당', '카페', '순대,순댓국', 'PC방', '바나프레소', '전처리필요'], dtype=object)

In [9]:
# 필요없는 카테고리(카페, 술, 출장요리 등등) 제거
제외 = ['바나프레소','이자카야','바(BAR)','카페,디저트','요리주점','장소대여','카페','PC','예식장']
for i in 제외 :
    res_info_df = res_info_df.drop(res_info_df[res_info_df['업태구분'] == i].index)

len(res_info_df)

20

In [10]:
res_info_df

,식당이름,업태구분,주소,메뉴,가격,리뷰,검색어
1,뢰벤돈까스,양식,서울 강남구 테헤란로 124 삼원타워 지하1층 B102호,등심돈까스,"9,000원",302,테헤란로 124 뢰벤 돈까스
2,생강김밥 본점,김밥,서울 강남구 남부순환로 2738 상일빌딩,문어김밥,"4,800원",2032,남부순환로 2738 생강김밥
3,삼성원조양곱창,"곱창,막창,양",서울 강남구 학동로101길 7 승창빌딩 1층,특양,"30,000원",315,"학동로101길 7, 삼성양곱창"
5,다미가정식백반,"백반,가정식",서울 강남구 개포로20길 16,가정식백반,"9,000원",53,개포로20길 16 다미
6,피자헛 삼성중앙점,피자,서울 강남구 봉은사로73길 19,(오)LA BBQ 불고기 L,"22,900원29,900원",393,"봉은사로73길 19, 한국피자헛"
8,목구멍 신논현점,"육류,고기요리",서울 강남구 봉은사로2길 16,미박삼겹살(150g),"13,000원",1210,봉은사로2길 16 목구멍
9,길목 신관,돼지고기구이,서울 강남구 봉은사로 504 진양빌딩 1층,투뿔목살,"18,000원",54,"봉은사로 504, 길목 신관"
11,집밥푸드,한식뷔페,서울 강남구 선릉로 28,메뉴없음,가격없음,3,"선릉로 28, 집밥푸드"
12,더 키푸드,뷔페,서울 강남구 선릉로 823 6층,정찬,"10,000원",87,"선릉로 823, 더키푸드"
13,정육점 아저씨의 투잡 대치지점,한식,서울 강남구 삼성로81길 43,메뉴없음,가격없음,1,"삼성로81길 43, 정육점 아저씨의 투잡"


In [11]:
# ## 한식 하나 더 만들어서 정렬되는지 확인하려고 만들었음 

# res_info_df.loc[df_test_result['업태구분']=='한식뷔페','업태구분' ] = '한식'

In [15]:
res_df_groupby = res_info_df.groupby('업태구분').apply(lambda x : x.sort_values(by='리뷰',ascending=False))


In [16]:
res_df_groupby

식당이름     업태구분  \
업태구분                                          
PC방     19           제로백PC 매직엔 개포점      PC방   
곱창,막창,양 22                   만득이곱창  곱창,막창,양   
        3                  삼성원조양곱창  곱창,막창,양   
김밥      2                  생강김밥 본점       김밥   
돈가스     14                      혼돈      돈가스   
돼지고기구이  9                    길목 신관   돼지고기구이   
백반,가정식  5                  다미가정식백반   백반,가정식   
뷔페      12                   더 키푸드       뷔페   
순대,순댓국  18           계경순대국 수서역벤처빌점   순대,순댓국   
양식      1                    뢰벤돈까스       양식   
육류,고기요리 8                 목구멍 신논현점  육류,고기요리   
        24             딸부자네불백 신사본점  육류,고기요리   
일식당     16  그랜드 인터컨티넨탈 서울 파르나스 하코네      일식당   
전처리필요   25       역삼로 607, 그랜드힐 컨벤션    전처리필요   
        27           테헤란로20길 18, 비    전처리필요   
        29      테헤란로20길 19, 로얄 위스키    전처리필요   
피자      6                피자헛 삼성중앙점       피자   
한식      23                   원두막식당       한식   
        11                    집밥푸드       한식   
        13        정육점 아저씨의 투잡 대치지점       한식   

                                               주소               메뉴  \
업태구분                                                                 
PC방     19                      서울 강남구 개포로 223 지층             메뉴없음   
곱창,막창,양 22            서울 강남구 강남대로124길 36 1층 만득이곱창      야채곱창(1~2인분)   
        3                서울 강남구 학동로101길 7 승창빌딩 1층               특양   
김밥      2                  서울 강남구 남부순환로 2738 상일빌딩             문어김밥   
돈가스     14                 서울 강남구 언주로115길 15 101호             멘치카츠   
돼지고기구이  9                 서울 강남구 봉은사로 504 진양빌딩 1층             투뿔목살   
백반,가정식  5                        서울 강남구 개포로20길 16            가정식백반   
뷔페      12                      서울 강남구 선릉로 823 6층               정찬   
순대,순댓국  18    서울 강남구 밤고개로1길 10 수서현대벤쳐빌 지하 2층 202호              순대국   
양식      1         서울 강남구 테헤란로 124 삼원타워 지하1층 B102호            등심돈까스   
육류,고기요리 8                        서울 강남구 봉은사로2길 16      미박삼겹살(150g)   
        24                     서울 강남구 강남대로158길 21              소불백   
일식당     16  서울 강남구 테헤란로 521 그랜드 인터컨티넨탈 서울 파르나스 1층               중식   
전처리필요   25                                  전처리필요             메뉴없음   
        27                                  전처리필요             메뉴없음   
        29                                  전처리필요             메뉴없음   
피자      6                       서울 강남구 봉은사로73길 19  (오)LA BBQ 불고기 L   
한식      23                     서울 강남구 강남대로152길 67           닭볶음탕 대   
        11                          서울 강남구 선릉로 28             메뉴없음   
        13                       서울 강남구 삼성로81길 43             메뉴없음   

                        가격    리뷰                       검색어  
업태구분                                                        
PC방     19            가격없음    86          개포로 223, 매직엔 PC방  
곱창,막창,양 22         16,000원   321        강남대로124길 36, 만득이곱창  
        3          30,000원   315          학동로101길 7, 삼성양곱창  
김밥      2           4,800원  2032           남부순환로 2738 생강김밥  
돈가스     14         13,000원   114            언주로115길 15, 혼돈  
돼지고기구이  9          18,000원    54           봉은사로 504, 길목 신관  
백반,가정식  5           9,000원    53              개포로20길 16 다미  
뷔페      12         10,000원    87             선릉로 823, 더키푸드  
순대,순댓국  18          8,000원   371  밤고개로1길 10, 계경순대국수서현대벤쳐빌점  
양식      1           9,000원   302           테헤란로 124 뢰벤 돈까스  
육류,고기요리 8          13,000원  1210             봉은사로2길 16 목구멍  
        24         10,000원   919        강남대로158길 21 딸부자네불백  
일식당     16              변동   210             테헤란로 521, 하꼬네  
전처리필요   25            가격없음     0         역삼로 607, 그랜드힐 컨벤션  
        27            가격없음     0             테헤란로20길 18, 비  
        29            가격없음     0        테헤란로20길 19, 로얄 위스키  
피자      6   22,900원29,900원   393         봉은사로73길 19, 한국피자헛  
한식      23         25,000원   261           강남대로152길 67 원두막  
        11            가격없음     3              선릉로 28, 집밥푸드  
        13            가격없음     1    삼성로81길 43, 정육점 아저씨의 투잡

In [14]:

# # 예제 데이터프레임 생성
# test = pd.DataFrame({
#     '업태구분': ['식당', '식당', '카페', '카페', '호텔', '호텔'],
#     '리뷰': [5, 4, 3, 2, 1, 6]
# })

# # 상위 n개를 뽑는 함수 정의
# def top_n(group, n=2):
#     return group.head(n)

# # '업태구분' 칼럼을 기준으로 그룹화하고, 각 그룹별로 정렬한 후 상위 2개의 데이터를 뽑아 새로운 데이터프레임으로 생성
# test_groupby = test.groupby('업태구분', group_keys=False).apply(lambda x: x.sort_values(by='리뷰', ascending=False)).groupby('업태구분').apply(top_n, n=2)

# print(test_groupby)

       업태구분  리뷰
업태구분           
식당   0   식당   5
     1   식당   4
카페   2   카페   3
     3   카페   2
호텔   5   호텔   6
     4   호텔   1


In [29]:
res_df_groupby

식당이름     업태구분  \
업태구분                                          
PC방     19           제로백PC 매직엔 개포점      PC방   
곱창,막창,양 22                   만득이곱창  곱창,막창,양   
        3                  삼성원조양곱창  곱창,막창,양   
김밥      2                  생강김밥 본점       김밥   
돈가스     14                      혼돈      돈가스   
돼지고기구이  9                    길목 신관   돼지고기구이   
백반,가정식  5                  다미가정식백반   백반,가정식   
뷔페      12                   더 키푸드       뷔페   
순대,순댓국  18           계경순대국 수서역벤처빌점   순대,순댓국   
양식      1                    뢰벤돈까스       양식   
육류,고기요리 8                 목구멍 신논현점  육류,고기요리   
        24             딸부자네불백 신사본점  육류,고기요리   
일식당     16  그랜드 인터컨티넨탈 서울 파르나스 하코네      일식당   
전처리필요   23         강남대로152길 67 원두막    전처리필요   
주류      29                   로얄위스키       주류   
피자      6                피자헛 삼성중앙점       피자   
한식      11                    집밥푸드       한식   
        13        정육점 아저씨의 투잡 대치지점       한식   

                                               주소           메뉴       가격    리뷰  \
업태구분                                                                            
PC방     19                      서울 강남구 개포로 223 지층         메뉴없음     가격없음    86   
곱창,막창,양 22            서울 강남구 강남대로124길 36 1층 만득이곱창  야채곱창(1~2인분)  16,000원   319   
        3                서울 강남구 학동로101길 7 승창빌딩 1층           특양  30,000원   315   
김밥      2                  서울 강남구 남부순환로 2738 상일빌딩         문어김밥   4,800원  2029   
돈가스     14                 서울 강남구 언주로115길 15 101호         멘치카츠  13,000원   113   
돼지고기구이  9                 서울 강남구 봉은사로 504 진양빌딩 1층         투뿔목살  18,000원    54   
백반,가정식  5                        서울 강남구 개포로20길 16        가정식백반   9,000원    53   
뷔페      12                      서울 강남구 선릉로 823 6층           정찬  10,000원    87   
순대,순댓국  18    서울 강남구 밤고개로1길 10 수서현대벤쳐빌 지하 2층 202호          순대국   8,000원   371   
양식      1         서울 강남구 테헤란로 124 삼원타워 지하1층 B102호        등심돈까스   9,000원   302   
육류,고기요리 8                        서울 강남구 봉은사로2길 16  미박삼겹살(150g)  13,000원  1147   
        24                     서울 강남구 강남대로158길 21          소불백  10,000원   919   
일식당     16  서울 강남구 테헤란로 521 그랜드 인터컨티넨탈 서울 파르나스 1층           중식       변동   210   
전처리필요   23                                  전처리필요         메뉴없음     가격없음     0   
주류      29            서울 강남구 테헤란로20길 19 지상1층 103호         메뉴없음     가격없음     8   
피자      6                       서울 강남구 봉은사로73길 19    핫소스(2.5g)     100원   392   
한식      11                          서울 강남구 선릉로 28         메뉴없음     가격없음     3   
        13                       서울 강남구 삼성로81길 43         메뉴없음     가격없음     1   

                                 검색어  
업태구분                                  
PC방     19          개포로 223, 매직엔 PC방  
곱창,막창,양 22        강남대로124길 36, 만득이곱창  
        3           학동로101길 7, 삼성양곱창  
김밥      2            남부순환로 2738 생강김밥  
돈가스     14            언주로115길 15, 혼돈  
돼지고기구이  9            봉은사로 504, 길목 신관  
백반,가정식  5               개포로20길 16 다미  
뷔페      12             선릉로 823, 더키푸드  
순대,순댓국  18  밤고개로1길 10, 계경순대국수서현대벤쳐빌점  
양식      1            테헤란로 124 뢰벤 돈까스  
육류,고기요리 8              봉은사로2길 16 목구멍  
        24        강남대로158길 21 딸부자네불백  
일식당     16             테헤란로 521, 하꼬네  
전처리필요   23           강남대로152길 67 원두막  
주류      29        테헤란로20길 19, 로얄 위스키  
피자      6          봉은사로73길 19, 한국피자헛  
한식      11              선릉로 28, 집밥푸드  
        13    삼성로81길 43, 정육점 아저씨의 투잡

In [ ]:
res_df_groupby.to_csv('매장_업태별_결과.csv')

,식당이름,업태구분,주소,메뉴,가격
0,정식당,한식,서울 강남구 강남대로114길 18 . 1층 정식당,소불고기주물럭,"16,000원"
1,정식당,한식,서울 강남구 강남대로114길 18 . 1층 정식당,소불고기주물럭,"16,000원"
2,채선당 샤브보트 강남역점,샤브샤브,서울 강남구 강남대로 378 . 지상1층,소고기 샤브샤브,"12,900원"
3,수아당 갤러리아백화점 명품관 WEST점,김밥,서울 강남구 압구정로 343,유부월남쌈,"18,000원"
4,알찬한끼,종합분식,서울 강남구 역삼로 204 삼환아르누보 지하1층 107호,알찬도시락,"5,000원"
5,세계관 삼성본점,한식,서울 강남구 삼성로96길 11 2층,투뿔 한우 곱도리탕,"32,000원"
6,분식을품다 한티점,종합분식,서울 강남구 역삼로 409 1층 102호,쌀 떡볶이,"5,900원"
7,식당민홍,일식당,서울 강남구 논현로146길 11 2층,오마카세 코스,"122,000원"
8,제주그집,한식,서울 강남구 논현로8길 11 지상1층,냉동삼겹살(150g),"14,000원"
9,만수정 강남점,"장어,먹장어요리",서울 강남구 강남대로 628 비율라빌딩 1층,민물장어 1kg ( 650~680g),"90,000원"
